In [ ]:
import numpy as np
from google.colab.patches import cv2_imshow
import argparse
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
mode = "display"
#model creation
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense (1024, activation='relu'))
model.add(Dropout (0.5))
model.add(Dense(7, activation='softmax'))

In [ ]:
def emotion_recog(frame):
  model.load_weights('model.h5')

  cv2.ocl.setUseOpenCL(False)
  emotion_dict = {0: "Angry", 1: "Disgusted", 2:"Fearful", 3:"Happy", 4:"Neutral",
  5: "Sad", 6:"Suprised"}
  facecasc = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = facecasc.detectMultiScale( gray, scaleFactor=1.3, minNeighbors=5)

  for (x,y,w,h) in faces:
    cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255,0,255), 3)
    roi_gray = gray[y:y+h, x:x + w]
    cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (48,48)), -1), 0)
    prediction = model.predict(cropped_img)
    maxindex = int(np.argmax(prediction))
    cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)

    return frame

In [ ]:
import cv2
cap = cv2.VideoCapture('/content/Input_video.mp4')
ret, frame = cap.read()
frame_height, frame_width,_ = frame.shape
size = (frame_width, frame_height)
out = cv2.VideoWriter('Output_video.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

print("Processing Video..")

while cap.isOpened():
  ret, frame = cap.read()

  if not ret:
    out.release()
    break
  output = emotion_recog(frame)
  out.write(output)
out.release()
print("Done processing video")

Processing Video..
1/1 [==============================] - 0s 24ms/step
Done processing video
